In [51]:
import cv2
import numpy as np

ref = cv2.imread("cereal.jpg", 0)
# 1.1 Matching
sift = cv2.SIFT_create()

def show(Img, window_name='image'):
    cv2.namedWindow(window_name,cv2.WINDOW_NORMAL)
    cv2.imshow(window_name, Img)
    cv2.waitKey(0)
    # closing all open windows
    cv2.destroyAllWindows()
    

In [52]:
#1.1 Matching steps 1-6 function lowes return key points, descriptors and lowe's ratio matches


def lowes(img1, img2):
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)

    good_matches = []
    listA = []
    for m, n in matches:

        if m.distance < 0.8 * n.distance:  # ratio test
            good_matches.append([m])
            listA.append(m)

    img3 = cv2.drawMatchesKnn(img1, kp1, img2, kp2, good_matches, None, flags=2)

    return kp1, kp2, des1, des2, listA




In [53]:

#1.2 Affine Transformation 1-8 rotate an image
    

# load in image grayscale
ref = cv2.imread("cereal.jpg", 0)
# rotate image (default -25 degrees) and scale it default is set to 0.6)
def rotate_image(image, rotation=25, scaling=0.6):
    #TODO if not grayscale convert to grayscale
    (h, w) = image.shape[:2]
    image_center = ((w / 2), (h / 2))
    rotation_matrix = cv2.getRotationMatrix2D(image_center, rotation, scaling)
    rslt = cv2.warpAffine(image, rotation_matrix, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return rslt


img_rotated = rotate_image(ref)

#displaly output of rotated image
show(ref, "Reference Image")
show(img_rotated, "Rotated Image")




In [54]:
cereal = cv2.imread("cereal.jpg")
#accepts color image, 
def Affine_Transform(image):
    
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    img_rotated = rotate_image(gray)

    # feature detection function
    kp1, kp2, des1, des2, lowe_matches = lowes(ref, img_rotated)

    # 4 format points

    ref_pts = np.float32([kp1[m.queryIdx].pt for m in lowe_matches]).reshape(-1, 1, 2)

    img_pts = np.float32([kp2[m.trainIdx].pt for m in lowe_matches]).reshape(-1, 1, 2)

    # 5 retrieve Matrix
    # cv2.estimateAffinePartial2D(from_pts, to_pts)[0]
    transform = cv2.estimateAffinePartial2D(ref_pts, img_pts)[0]

    # 6 modify reference image just loaded in colour image
    modified_ref= image
    #original colored image will be used as the modified reference image
    modified_ref = cv2.warpAffine(modified_ref, transform, modified_ref.shape[1::-1], flags=cv2.INTER_LINEAR)
    img_rotated = cv2.cvtColor(img_rotated,cv2.COLOR_GRAY2RGB)
    result = cv2.addWeighted(modified_ref,0.7,img_rotated,0.3,0)
    return result


result =Affine_Transform(cereal)
show(result, "overlaid Affine")

In [55]:
#1.2 Perspective Transform 
ref = cv2.imread("cereal.jpg")

cereal_l = cv2.imread("cereal_l.jpg",0)
cereal_r =cv2.imread("cereal_r.jpg",0)
cereal_tl =cv2.imread("cereal_tl.jpg",0)
show(perspective, "input Image for perspective")
show(ref, "referance Image")




In [57]:
def perspective_Transform(ref_image, pers_image):
    
    
    kp1, kp2, des1, des2, lowe_matches = lowes(ref_image, pers_image)

    ref_pts = np.float32([kp1[m.queryIdx].pt for m in lowe_matches]).reshape(-1, 1, 2)

    img_pts = np.float32([kp2[m.trainIdx].pt for m in lowe_matches]).reshape(-1, 1, 2)


    M, _ = cv2.findHomography(ref_pts, img_pts, cv2.RANSAC, 5.0)
    
    img_size= (pers_image.shape[1],pers_image.shape[0])
    

    #original first is image, second is matrix ,3 is the size of the matrix
    modified_ref = cv2.warpPerspective(ref_image, M, img_size)


    test_color = cv2.cvtColor(pers_image,cv2.COLOR_GRAY2RGB)
    
    result = cv2.addWeighted(modified_ref,0.5,test_color,0.3,0)

    show(result)
    
    
perspective_Transform(ref,cereal_l)
perspective_Transform(ref,cereal_r)

perspective_Transform(ref,cereal_tl)

(1104, 828, 3)
(1104, 828, 3)
(1104, 828, 3)
(1104, 828, 3)
(1104, 828, 3)
(1104, 828, 3)
